In [ ]:
# !pip install pygame


#### Infos sobre palavras
*Palavras já vistas*
- Quantas vezes você viu essa palavra
- Nivel de conhecimento sobre a palavra
    - Se ja viu 100 vezes e das 100 ccertou 80% você domina essa palavra | se 10% você tem muita dificuldade nessa palavra


*Palavras não vistas*

---

#### Jogos
- jogo da forca

- Inclusões
    - Escrever, escultar e adivinhar
    - Falar a palavra/frase



---

#### Requisitos
- Ao ver uma palavra ela já dev

In [1]:
import tkinter as tk
from tkinter import messagebox
from PIL import Image, ImageTk
import os
import json
import pygame
from pathlib import Path

# Inicializa o áudio
pygame.mixer.init()

# Caminho base dos dados
DATA_PATH = Path(r"data_organize")

# Funções para obter os caminhos
def get_list_category():
    return [DATA_PATH / path for path in os.listdir(DATA_PATH)]

def get_list_sub_category(path_category):
    return [path_category / path for path in os.listdir(path_category)]

def get_list_words(path_sub_category):
    return [path_sub_category / path for path in os.listdir(path_sub_category)]

def get_files_from_word(path_word):
    return [path_word / path for path in os.listdir(path_word)]

# Monta uma lista de palavras com seus respectivos diretórios
def construir_lista_palavras(category=None, sub_category_name=None):
    palavras = []
    for categoria in get_list_category():
        for sub_categoria in get_list_sub_category(categoria):
            
            sub_categoria = Path(sub_categoria)
            if sub_category_name:
                return [palavra for palavra in get_list_words(sub_categoria)]

            for palavra in get_list_words(sub_categoria):
                palavras.append(palavra)
    return palavras


lista_palavras = construir_lista_palavras(sub_category_name="decoração")
indice_palavra = 0

def carregar_dados(indice):
    palavra_path = lista_palavras[indice]
    arquivos = os.listdir(palavra_path)

    try:
        with open(palavra_path / "text.json", "r", encoding="utf-8") as f:
            texto_info = json.load(f)
    except:
        texto_info = {}

    imagens = sorted(
        [palavra_path / arq for arq in arquivos if arq.startswith("recorte_") and arq.endswith(".jpg") and "recorte_4" not in arq],
        key=lambda x: int(x.stem.split("_")[1])
    )
    audio_path = palavra_path / "audio.wav"
    fonetica_path = palavra_path / "recorte_4.jpg"
    return imagens, texto_info, audio_path, fonetica_path

# Interface Gráfica
janela = tk.Tk()
janela.title("Fast English")

# Elementos de interface
top = tk.Frame(janela)
top.pack()
tk.Button(top, text="< (sair)", command=janela.quit).pack(side="left")
progresso_label = tk.Label(top, text="Progresso...")
progresso_label.pack(side="left", padx=20)

mid = tk.Frame(janela)
mid.pack()
btn_ant = tk.Button(mid, text="<")
btn_ant.grid(row=0, column=0)
img_label = tk.Label(mid)
img_label.grid(row=0, column=1)
btn_prox = tk.Button(mid, text=">")
btn_prox.grid(row=0, column=2)

traducao_label = tk.Label(janela, text="Tradução:")
traducao_label.pack()
tipo_label = tk.Label(janela, text="Tipo:")
tipo_label.pack()
palavra_label = tk.Label(janela, text="Palavra:")
palavra_label.pack()
fonetica_label = tk.Label(janela)
fonetica_label.pack()
btn_audio = tk.Button(janela, text="🔊 Ouvir")
btn_audio.pack(pady=5)

# Atualização da interface
def atualizar_interface():
    global imagens, texto_info, audio_path, fonetica_path

    progresso_label.config(text=f"{indice_palavra + 1}/{len(lista_palavras)}")

    imagens, texto_info, audio_path, fonetica_path = carregar_dados(indice_palavra)

    if imagens:
        img = Image.open(imagens[0]).resize((200, 200))
        photo = ImageTk.PhotoImage(img)
        img_label.config(image=photo)
        img_label.image = photo

    traducao_label.config(text=f"Tradução: {texto_info.get('recorte_1.jpg', '')}")
    tipo_label.config(text=f"Tipo: {texto_info.get('recorte_2.jpg', '')}")
    palavra_label.config(text=f"Palavra: {texto_info.get('recorte_3.jpg', '')}")

    if fonetica_path.exists():
        fon_img = Image.open(fonetica_path).resize((150, 50))
        fon_photo = ImageTk.PhotoImage(fon_img)
        fonetica_label.config(image=fon_photo)
        fonetica_label.image = fon_photo

def tocar_audio():
    if audio_path.exists():
        pygame.mixer.music.load(str(audio_path))
        pygame.mixer.music.play()
    else:
        messagebox.showinfo("Áudio", "Áudio não encontrado.")

def proximo():
    global indice_palavra
    if indice_palavra < len(lista_palavras) - 1:
        indice_palavra += 1
        atualizar_interface()

def anterior():
    global indice_palavra
    if indice_palavra > 0:
        indice_palavra -= 1
        atualizar_interface()

# Conecta botões
btn_prox.config(command=proximo)
btn_ant.config(command=anterior)
btn_audio.config(command=tocar_audio)

# Inicializa interface
atualizar_interface()
janela.mainloop()
janela.destroy()


pygame 2.6.1 (SDL 2.28.4, Python 3.12.10)
Hello from the pygame community. https://www.pygame.org/contribute.html


TclError: can't invoke "destroy" command: application has been destroyed